# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [2]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [3]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [3]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [4]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [5]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

In [6]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [7]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [8]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   gender               14825 non-null  object        
 1   age                  17000 non-null  int64         
 2   id                   17000 non-null  object        
 3   became_member_on     17000 non-null  datetime64[ns]
 4   income               14825 non-null  float64       
 5   bec_memb_year_month  17000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 797.0+ KB


In [9]:
portfolio = portfolio_raw.copy()
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [79]:
# Standardisation of column names
transcript = transcript_raw.copy(deep=True)
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalizing the value column and create a new dataframe
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id)
transcript_b['ofr_id_short'] = transcript_b['ofr_id_short'].fillna('tran')
transcript_b['amount'] = transcript_b['amount'].fillna(0)
transcript_b['reward'] = transcript_b['reward'].fillna(0)

transcript_b = transcript_b.drop(columns = ['offer_id'])

transcript_b['tag'] = (
    transcript_b.groupby(['person', 'ofr_id_short', 'event'])
    .cumcount()
)

transactions = transcript_b[transcript_b['ofr_id_short'] == 'tran'].copy()
events = transcript_b[transcript_b['ofr_id_short'] != 'tran'].copy()

events = events.groupby(['person', 'ofr_id_short', 'event', 'tag']).agg(
    time=('time', 'mean'),
    reward=('reward', 'sum'),
).unstack([2]).reset_index()

events.columns = ['_'.join(col).strip('_') for col in events.columns.to_flat_index()]

events = events.drop(columns=['reward_offer_received',	'reward_offer_viewed'])

events['time_offer_completed'] = events['time_offer_completed'].fillna(np.inf)
events['time_offer_received'] = events['time_offer_received'].fillna(np.inf)
events['time_offer_viewed'] = events['time_offer_viewed'].fillna(np.inf)

events['to_vr'] = events['time_offer_viewed'] - events['time_offer_received']
events['to_cv'] = events['time_offer_completed'] - events['time_offer_viewed'] 
events['to_cr'] = events['time_offer_completed'] - events['time_offer_received']

events['curiosity_vr'] = (1 / (events['to_vr'] + 1)) * 100
events['eagerness_cv'] = (1 / ((events['to_cv']) + 1)) * 100
events['overall_cr'] = (1 / (events['to_cr'] + 1)) * 100

events = events.drop(columns=['time_offer_completed', 'time_offer_received', 'time_offer_viewed', 'to_vr', 'to_cv', 'to_cr'	]).round(1)



transac_person_sumary = transactions.groupby(['person']).agg(
    # tran_count=('amount', 'count'),
    tran_amoun_tot =('amount', 'sum'),
    tran_amoun_mean=('amount', 'mean'),
    tran_amoun_max=('amount', 'max'),
    tran_amoun_min=('amount', 'min'),
    # tran_amount_std = ('amount', 'std')
).reset_index().round(1)

event_count = transcript_b.groupby(['person', 'event']).agg(
    cnt = ('event', 'count'),
    avg_time = ('time', 'mean'),
    max_time = ('time', 'max'),
    min_time = ('time', 'min')
    
    ).unstack(1).reset_index()

event_count.columns = ['_'.join(col).strip('_') for col in event_count.columns.to_flat_index()]

# events_merged = events.merge(transactions, left_on=['person', 'time_offer_completed'], right_on=['person', 'time'], how='left')

person_activity = events.merge(transac_person_sumary, on='person', how='left')

person_activity = person_activity.merge(event_count, on='person', how='left').fillna(0)

person_activity_profile = person_activity.merge(profile, left_on='person', right_on='id', how='left').drop(columns=['id'])



person_activity_profile.to_csv('person_activity_profile.csv')

person_activity_profile



,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,tran_amoun_tot,tran_amoun_mean,tran_amoun_max,...,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction,gender,age,became_member_on,income,bec_memb_year_month
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,2.7,0.0,0.0,127.6,16.0,28.2,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,2.0,2.7,-9.1,4.0,127.6,16.0,28.2,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,4.0,0.0,0.0,127.6,16.0,28.2,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,5.0,2.0,-2.4,14.3,127.6,16.0,28.2,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,2.0,0.0,-0.0,100.0,127.6,16.0,28.2,...,696.0,414.0,168.0,192.0,228.0,M,33,2017-04-21,72000.0,2017-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,5.0,14.3,4.0,3.2,226.1,15.1,23.3,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,2.0,14.3,1.8,1.6,226.1,15.1,23.3,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,2.0,5.3,3.2,2.0,226.1,15.1,23.3,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,2.0,14.3,100.0,14.3,226.1,15.1,23.3,...,648.0,60.0,0.0,6.0,60.0,F,45,2016-11-25,62000.0,2016-11


In [80]:
person_activity_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   person                    76277 non-null  object        
 1   ofr_id_short              76277 non-null  object        
 2   tag                       76277 non-null  int64         
 3   reward_offer_completed    76277 non-null  float64       
 4   curiosity_vr              76277 non-null  float64       
 5   eagerness_cv              76277 non-null  float64       
 6   overall_cr                76277 non-null  float64       
 7   tran_amoun_tot            76277 non-null  float64       
 8   tran_amoun_mean           76277 non-null  float64       
 9   tran_amoun_max            76277 non-null  float64       
 10  tran_amoun_min            76277 non-null  float64       
 11  cnt_offer_completed       76277 non-null  float64       
 12  cnt_offer_received

In [67]:
person_activity_profile[person_activity_profile['tran_amount_std'].isna()]

,person,ofr_id_short,tag,reward_offer_completed,curiosity_vr,eagerness_cv,overall_cr,tran_amoun_tot,tran_amoun_mean,tran_amoun_max,...,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction,gender,age,became_member_on,income,bec_memb_year_month
241,00ee2ca6421c4af0aeca60a1b3e00f6c,ofr_B,0,0.0,1.1,0.0,0.0,4.0,4.0,4.0,...,708.0,NaN,168.0,174.0,708.0,M,60,2017-08-16,74000.0,2017-08
242,00ee2ca6421c4af0aeca60a1b3e00f6c,ofr_G,0,0.0,14.3,0.0,0.0,4.0,4.0,4.0,...,708.0,NaN,168.0,174.0,708.0,M,60,2017-08-16,74000.0,2017-08
243,00ee2ca6421c4af0aeca60a1b3e00f6c,ofr_H,0,0.0,14.3,0.0,0.0,4.0,4.0,4.0,...,708.0,NaN,168.0,174.0,708.0,M,60,2017-08-16,74000.0,2017-08
244,00ee2ca6421c4af0aeca60a1b3e00f6c,ofr_J,0,0.0,5.3,0.0,0.0,4.0,4.0,4.0,...,708.0,NaN,168.0,174.0,708.0,M,60,2017-08-16,74000.0,2017-08
486,01d7da27b8934ba1b3602a0153e4415f,ofr_D,0,0.0,1.1,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,168.0,NaN,F,44,2018-03-12,76000.0,2018-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76209,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_C,0,0.0,14.3,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,12.0,NaN,M,65,2018-02-15,52000.0,2018-02
76210,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_I,0,0.0,7.7,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,12.0,NaN,M,65,2018-02-15,52000.0,2018-02
76211,ffd810f301ad42d6bfe98c4ba6ee4125,ofr_J,0,0.0,1.2,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,12.0,NaN,M,65,2018-02-15,52000.0,2018-02
76238,ffed75d3abc64b488982f50ed12878b5,ofr_A,0,0.0,7.7,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,18.0,NaN,M,20,2018-03-15,44000.0,2018-03


In [70]:
transcript_b[transcript_b['person'] == 'ffed75d3abc64b488982f50ed12878b5']

,person,event,time,amount,reward,ofr_id_short,tag
2970,ffed75d3abc64b488982f50ed12878b5,offer_received,0,0.0,0.0,ofr_B,0
20733,ffed75d3abc64b488982f50ed12878b5,offer_viewed,18,0.0,0.0,ofr_B,0
113829,ffed75d3abc64b488982f50ed12878b5,offer_received,336,0.0,0.0,ofr_A,0
130780,ffed75d3abc64b488982f50ed12878b5,offer_viewed,348,0.0,0.0,ofr_A,0


In [61]:
unique_transactions_persons = set(transactions['person'])
unique_events_persons = set(events['person'].unique())

common_persons_count = len(unique_events_persons.intersection(unique_transactions_persons))
print(common_persons_count)

16572


In [62]:
len(unique_transactions_persons)

16578

In [63]:
len(unique_events_persons)

16994

In [64]:
len(unique_events_persons) - len(unique_transactions_persons)

416

In [57]:
transactions['person'].unique()

array(['02c083884c7d45b39cc68e1314fec56c',
       '9fa9ae8f57894cc9a3b8a9bbe0fc1b2f',
       '54890f68699049c2a04d415abc25e717', ...,
       '7718656997f3453db0f5aeca9cd35240',
       '54463e5d95124b7fb3133fc1eae71952',
       '0c027f5f34dd4b9eba0a25785c611273'], shape=(16578,), dtype=object)

In [42]:
transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        306534 non-null  object 
 1   event         306534 non-null  object 
 2   time          306534 non-null  int64  
 3   amount        306534 non-null  float64
 4   reward        306534 non-null  float64
 5   ofr_id_short  306534 non-null  object 
 6   tag           306534 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 16.4+ MB


In [41]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76277 entries, 0 to 76276
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   person                  76277 non-null  object 
 1   ofr_id_short            76277 non-null  object 
 2   tag                     76277 non-null  int64  
 3   reward_offer_completed  33579 non-null  float64
 4   curiosity_vr            76277 non-null  float64
 5   eagerness_cv            63354 non-null  float64
 6   overall_cr              76277 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 4.1+ MB


In [40]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138953 entries, 12654 to 306533
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        138953 non-null  object 
 1   event         138953 non-null  object 
 2   time          138953 non-null  int64  
 3   amount        138953 non-null  float64
 4   reward        138953 non-null  float64
 5   ofr_id_short  138953 non-null  object 
 6   tag           138953 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 8.5+ MB


In [39]:
transactions['amount'].value_counts(dropna=False)

amount
0.05       431
0.66       166
1.18       165
1.01       163
0.90       161
          ... 
40.83        1
1015.73      1
51.06        1
78.85        1
749.29       1
Name: count, Length: 5103, dtype: int64

In [16]:
transcript_b

,person,event,time,amount,reward,ofr_id_short,tag
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,0.00,0.0,ofr_D,0
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0.00,0.0,ofr_E,0
2,e2127556f4f64592b11af22de27a7932,offer_received,0,0.00,0.0,ofr_J,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,0.00,0.0,ofr_G,0
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,0.00,0.0,ofr_B,0
...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,0.0,tran,13
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,0.0,tran,1
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,0.0,tran,19
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,0.0,tran,12


In [51]:
# Check if all values in transactions['person'] are in events['person']
all_in_events = transactions['person'].isin(events['person']).all()

# Check if all values in events['person'] are in transactions['person']
all_in_transactions = events['person'].isin(transactions['person']).all()

print(f"All values in transactions['person'] are in events['person']: {all_in_events}")
print(f"All values in events['person'] are in transactions['person']: {all_in_transactions}")

All values in transactions['person'] are in events['person']: False
All values in events['person'] are in transactions['person']: False


In [28]:
events

,person,ofr_id_short,tag,time_offer_completed,time_offer_received,time_offer_viewed,reward_offer_completed
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,NaN,336.0,372.0,NaN
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,528.0,504.0,540.0,2.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,NaN,168.0,192.0,NaN
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,414.0,408.0,456.0,5.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,576.0,576.0,NaN,2.0
...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,198.0,168.0,174.0,5.0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,60.0,0.0,6.0,2.0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,384.0,336.0,354.0,2.0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,414.0,408.0,414.0,2.0


In [19]:
transac_person_sumary

,person,tran_amoun_tot,tran_amoun_mean,tran_amoun_max,tran_amoun_min,tran_amount_std
0,0009655768c64bdeb2e877511632db8f,127.6,16.0,28.2,8.6,6.6
1,00116118485d4dfda04fdbaba9a87b5c,4.1,1.4,3.2,0.2,1.6
2,0011e0d4e6b944f998e987f904e8c1e5,79.5,15.9,23.0,9.0,6.3
3,0020c2b971eb4e9188eac86d93036a77,196.9,24.6,33.9,17.2,6.3
4,0020ccbbb6d84e358d3414a3ff76cffd,154.0,12.8,20.1,6.8,4.3
...,...,...,...,...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,581.0,52.8,388.2,11.0,111.3
16574,fff7576017104bcc8677a8d63322b5e1,29.9,5.0,8.0,2.1,2.0
16575,fff8957ea8b240a6b5e634b6ee8eafcf,12.2,2.4,6.4,0.6,2.5
16576,fffad4f4828548d1b5583907f2e9906b,88.8,7.4,12.2,2.0,3.0


In [20]:
event_count

,person,cnt_offer_completed,cnt_offer_received,cnt_offer_viewed,cnt_transaction,avg_time_offer_completed,avg_time_offer_received,avg_time_offer_viewed,avg_time_transaction,max_time_offer_completed,max_time_offer_received,max_time_offer_viewed,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction
0,0009655768c64bdeb2e877511632db8f,3.0,5.0,4.0,8.0,506.0,398.4,390.0,543.00,576.0,576.0,540.0,696.0,414.0,168.0,192.0,228.0
1,00116118485d4dfda04fdbaba9a87b5c,NaN,2.0,2.0,3.0,NaN,372.0,423.0,408.00,NaN,576.0,630.0,474.0,NaN,168.0,216.0,294.0
2,0011e0d4e6b944f998e987f904e8c1e5,3.0,5.0,5.0,5.0,468.0,283.2,298.8,451.20,576.0,504.0,516.0,654.0,252.0,0.0,6.0,132.0
3,0020c2b971eb4e9188eac86d93036a77,3.0,5.0,3.0,8.0,358.0,283.2,366.0,348.75,510.0,504.0,660.0,708.0,54.0,0.0,12.0,54.0
4,0020ccbbb6d84e358d3414a3ff76cffd,3.0,4.0,4.0,12.0,400.0,354.0,376.5,375.00,600.0,504.0,582.0,672.0,222.0,168.0,168.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,fff3ba4757bd42088c044ca26d73817a,3.0,6.0,3.0,11.0,234.0,332.0,252.0,246.00,528.0,576.0,540.0,552.0,6.0,0.0,6.0,6.0
16996,fff7576017104bcc8677a8d63322b5e1,3.0,5.0,4.0,6.0,460.0,331.2,285.0,392.00,594.0,576.0,522.0,696.0,192.0,0.0,24.0,36.0
16997,fff8957ea8b240a6b5e634b6ee8eafcf,NaN,3.0,2.0,5.0,NaN,496.0,546.0,379.20,NaN,576.0,660.0,576.0,NaN,408.0,432.0,18.0
16998,fffad4f4828548d1b5583907f2e9906b,3.0,4.0,4.0,12.0,380.0,288.0,337.5,323.50,588.0,576.0,666.0,678.0,36.0,0.0,6.0,36.0


In [21]:
person_activity[person_activity['person'] == 'e23280a162b34a94a7b45ce5d9b18b7a']  

,person,ofr_id_short,tag,time_offer_completed,time_offer_received,time_offer_viewed,reward_offer_completed,tran_amoun_tot,tran_amoun_mean,tran_amoun_max,...,avg_time_offer_viewed,avg_time_transaction,max_time_offer_completed,max_time_offer_received,max_time_offer_viewed,max_time_transaction,min_time_offer_completed,min_time_offer_received,min_time_offer_viewed,min_time_transaction
67547,e23280a162b34a94a7b45ce5d9b18b7a,ofr_B,0,168.0,168.0,312.0,10.0,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0
67548,e23280a162b34a94a7b45ce5d9b18b7a,ofr_D,0,390.0,336.0,360.0,5.0,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0
67549,e23280a162b34a94a7b45ce5d9b18b7a,ofr_D,1,444.0,408.0,NaN,5.0,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0
67550,e23280a162b34a94a7b45ce5d9b18b7a,ofr_E,0,90.0,0.0,84.0,5.0,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0
67551,e23280a162b34a94a7b45ce5d9b18b7a,ofr_H,0,NaN,576.0,618.0,NaN,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0
67552,e23280a162b34a94a7b45ce5d9b18b7a,ofr_J,0,504.0,504.0,NaN,2.0,305.3,27.8,45.7,...,343.5,308.181818,504.0,576.0,618.0,624.0,90.0,0.0,84.0,90.0


In [ ]:
person_activity[person_activity['person'] == '00116118485d4dfda04fdbaba9a87b5c']  

In [ ]:
person_activity[person_activity['person'] == '0011e0d4e6b944f998e987f904e8c1e5']  

In [ ]:
person_activity[person_activity['person'] == '0020c2b971eb4e9188eac86d93036a77']  

In [ ]:
sns.scatterplot(data=person_activity, x='time_offer_viewed', y='time_offer_completed', hue='ofr_id_short_x')

In [ ]:
sns.scatterplot(data=person_activity[(person_activity['person'] == '0020c2b971eb4e9188eac86d93036a77')], x='time_offer_viewed', y='time_offer_completed', hue='ofr_id_short_x') #(person_activity['ofr_id_short_x'] == 'ofr_B') & 

In [ ]:
transcript_b